# NiFi Processor Usage Analyzer - Multi-Flow Edition

This notebook analyzes NiFi processor execution counts across **multiple process groups** to identify unused or underutilized processors.

**Features:**
- Analyzes multiple flows from CSV input
- Fast execution count analysis (~5-10 seconds per flow)
- Snapshot mode with flow_name tracking
- Delta Lake integration with timestamp
- Standalone - no external files needed

**Setup:**
1. Upload CSV with flow definitions (id, flow_name)
2. Edit the configuration in Cell 3
3. Run all cells
4. View results in Delta table

In [ ]:
# Cell 1: Install Dependencies
%pip install requests rich --quiet
print("✓ Dependencies installed successfully!")

In [ ]:
# Cell 2: Import Libraries

import requests
import logging
from typing import Dict, List, Optional, Any
from datetime import datetime
from rich.console import Console
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, TextColumn, BarColumn

# Databricks-specific imports
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

# Disable SSL warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('nifi_analyzer')

# Initialize Rich console
console = Console()

print("✓ Libraries imported successfully!")

In [ ]:
# Cell 3: Configuration
# EDIT THESE VALUES FOR YOUR NIFI INSTANCE

CONFIG = {
    # NiFi Connection
    'nifi_url': 'https://thbnk01hdpnp002.th-bnk01.nxp.com:8443/nifi',
    'username': 'nxg16670',
    'password': 'your-password-here',  # ← EDIT THIS
    'verify_ssl': False,
    
    # Server Identifier (for tracking multiple NiFi servers)
    'server': 'prod-nifi-01',  # ← EDIT THIS (e.g., 'prod', 'dev', hostname)
    
    # Flow Definitions CSV
    # CSV Format: id,flow_name
    # Example:
    #   8c8677c4-29d6-36...,Production_Flow_1
    #   abc-123-def...,Development_Flow_2
    'flows_csv_path': '/dbfs/nifi_analysis/flows.csv',  # ← Path to your CSV
    
    # Snapshot Storage (Unity Catalog - 3-level naming)
    'enable_snapshots': True,
    'delta_table_path': 'main.default.nifi_processor_snapshots',  # catalog.schema.table
}

console.print("[green]✓ Configuration loaded![/green]")
console.print(f"  NiFi URL: {CONFIG['nifi_url']}")
console.print(f"  Username: {CONFIG['username']}")
console.print(f"  Server: {CONFIG['server']}")
console.print(f"  Flows CSV: {CONFIG['flows_csv_path']}")
console.print(f"  Delta table: {CONFIG['delta_table_path']}")
console.print(f"  Snapshots enabled: {CONFIG['enable_snapshots']}")

In [ ]:
# Cell 4: NiFi Client Class - RECURSIVE PROCESSOR EXTRACTION WITH PARENT GROUP INFO

class NiFiClient:
    """Client for interacting with Apache NiFi REST API."""
    
    def __init__(self, base_url: str, username: str, password: str, verify_ssl: bool = True):
        self.base_url = base_url.rstrip('/')
        if not self.base_url.endswith('/nifi'):
            self.base_url += '/nifi'
        self.api_url = f"{self.base_url}-api"
        self.verify_ssl = verify_ssl
        self.session = requests.Session()
        self.token = None
        self.username = username
        self.password = password
        self._authenticate(username, password)
        
    def _authenticate(self, username: str, password: str) -> None:
        """Authenticate with NiFi."""
        try:
            response = requests.post(
                f"{self.api_url}/access/token",
                data={'username': username, 'password': password},
                verify=self.verify_ssl
            )
            
            if response.status_code == 201:
                self.token = response.text
                self.session.headers.update({'Authorization': f'Bearer {self.token}'})
                logger.info("Successfully authenticated with token")
            else:
                logger.warning(f"Token auth failed with status {response.status_code}")
                logger.warning("Falling back to basic auth")
                from requests.auth import HTTPBasicAuth
                self.session.auth = HTTPBasicAuth(username, password)
        except Exception as e:
            logger.warning(f"Token auth error: {e}, falling back to basic auth")
            from requests.auth import HTTPBasicAuth
            self.session.auth = HTTPBasicAuth(username, password)
    
    def _request(self, method: str, endpoint: str, **kwargs) -> requests.Response:
        """Make authenticated request with 401 retry."""
        url = f"{self.api_url}/{endpoint.lstrip('/')}"
        kwargs.setdefault('verify', self.verify_ssl)
        
        response = self.session.request(method, url, **kwargs)
        
        # Handle 401 by re-authenticating once
        if response.status_code == 401:
            logger.warning("Received 401, attempting re-authentication")
            self._authenticate(self.username, self.password)
            response = self.session.request(method, url, **kwargs)
            if response.status_code == 401:
                raise Exception("Authentication failed: Unauthorized")
        
        response.raise_for_status()
        return response
    
    def get_process_group(self, group_id: str) -> Dict[str, Any]:
        """Get process group details including all processors."""
        response = self._request("GET", f"/flow/process-groups/{group_id}")
        return response.json()
    
    def get_process_group_status(self, group_id: str) -> Dict[str, Any]:
        """Get execution statistics for process group."""
        response = self._request("GET", f"/flow/process-groups/{group_id}/status?recursive=true")
        return response.json()
    
    def get_process_group_name(self, group_id: str) -> str:
        """
        Get the name of a process group by its ID.

        Args:
            group_id: Process group UUID

        Returns:
            Process group name, or group_id if fetch fails
        """
        try:
            response = self._request("GET", f"/flow/process-groups/{group_id}")
            data = response.json()
            group_name = data.get('processGroupFlow', {}).get('component', {}).get('name', group_id)
            logger.info(f"Fetched group name for {group_id}: {group_name}")
            return group_name
        except Exception as e:
            logger.warning(f"Failed to fetch group name for {group_id}: {str(e)}")
            return group_id  # Fallback to ID if name fetch fails
    
    def get_processor_statistics(self, group_id: str) -> List[Dict[str, Any]]:
        """
        Get processor-level statistics with IDs and parent groups using Status API.

        Uses Status API with recursive=true which returns nested processGroupStatusSnapshots.
        We recursively walk through all levels to extract ALL processors from deeply nested groups,
        along with their parent group IDs and names.

        Returns:
            List of processor dictionaries with IDs, metrics, and parent group info
        """
        # Get Status API with recursive=true ONCE
        status_data = self.get_process_group_status(group_id)

        # First pass: Recursively extract processors WITH groupId
        def extract_processors_recursive(pg_snapshot_data, depth=0):
            """Recursively extract processors from nested process group status snapshot."""
            processors = []

            # Extract processors at THIS level from processorStatusSnapshots array
            processor_snapshots = pg_snapshot_data.get('processorStatusSnapshots', [])
            
            logger.info(f"{'  ' * depth}Level {depth}: Found {len(processor_snapshots)} processors")

            for snapshot in processor_snapshots:
                # Each snapshot wraps a processorStatusSnapshot with the actual data
                proc_snap = snapshot.get('processorStatusSnapshot', {})

                processors.append({
                    'processor_id': proc_snap.get('id'),
                    'processor_name': proc_snap.get('name'),
                    'processor_type': proc_snap.get('type', '').split('.')[-1],
                    'parent_group_id': proc_snap.get('groupId'),  # NEW: Capture parent group ID
                    'flow_files_in': int(proc_snap.get('flowFilesIn', 0)),
                    'bytes_in': int(proc_snap.get('bytesIn', 0)),
                    'flow_files_out': int(proc_snap.get('flowFilesOut', 0)),
                    'bytes_out': int(proc_snap.get('bytesOut', 0)),
                    'tasks': int(proc_snap.get('taskCount', 0)),
                    'run_status': proc_snap.get('runStatus', 'Unknown')
                })

            # Recursively process nested child process groups
            # Note: processGroupStatusSnapshots (plural) contains wrapped processGroupStatusSnapshot (singular) objects
            child_group_snapshots = pg_snapshot_data.get('processGroupStatusSnapshots', [])
            if child_group_snapshots:
                logger.info(f"{'  ' * depth}Level {depth}: Found {len(child_group_snapshots)} child groups, recursing...")

            for child_group_snapshot in child_group_snapshots:
                # Unwrap the processGroupStatusSnapshot object
                child_pg_snapshot = child_group_snapshot.get('processGroupStatusSnapshot', {})
                child_processors = extract_processors_recursive(child_pg_snapshot, depth + 1)
                processors.extend(child_processors)

            return processors

        # Start recursion from root - need to navigate to aggregateSnapshot first
        pg_status = status_data.get('processGroupStatus', {})
        aggregate_snapshot = pg_status.get('aggregateSnapshot', {})
        all_processors = extract_processors_recursive(aggregate_snapshot)

        logger.info(f"Total processors extracted from all levels: {len(all_processors)}")

        # Second pass: Collect unique parent group IDs and batch fetch names
        parent_group_ids = set()
        for proc in all_processors:
            if proc.get('parent_group_id'):
                parent_group_ids.add(proc['parent_group_id'])

        logger.info(f"Fetching names for {len(parent_group_ids)} unique parent groups...")
        group_name_cache = {}
        for pg_id in parent_group_ids:
            group_name_cache[pg_id] = self.get_process_group_name(pg_id)

        # Third pass: Add parent group names to all processors
        for proc in all_processors:
            parent_id = proc.get('parent_group_id')
            proc['parent_group_name'] = group_name_cache.get(parent_id, parent_id) if parent_id else None

        logger.info(f"Returning {len(all_processors)} processor records with parent group info")
        return all_processors
    
    def close(self):
        """Close session."""
        self.session.close()

console.print("[green]✓ NiFiClient class defined (with parent group info)![/green]")

In [ ]:
# Cell 5: Multi-Flow Analyzer Class - PROCESSOR-LEVEL WITH PARENT GROUP INFO

class MultiFlowAnalyzer:
    """Analyzes multiple NiFi flows and stores PROCESSOR-LEVEL results in Delta Lake."""
    
    def __init__(self, client: NiFiClient, server: str = 'unknown'):
        self.client = client
        self.console = Console()
        self.server = server
        self.all_results = []
        self.snapshot_timestamp = datetime.now()
    
    def analyze_flow(self, flow_id: str, flow_name: str) -> Dict:
        """Analyze a single flow - processor level."""
        flow_results = {
            'flow_name': flow_name,
            'flow_id': flow_id,
            'processor_count': 0,
            'processors': []
        }
        
        try:
            # Get processor-level statistics with parent group info
            processors = self.client.get_processor_statistics(flow_id)
            flow_results['processor_count'] = len(processors)
            
            # Add metadata to each processor
            for proc in processors:
                flow_results['processors'].append({
                    # Metadata (4 fields)
                    'snapshot_timestamp': self.snapshot_timestamp,
                    'server': self.server,
                    'flow_name': flow_name,
                    'process_group_id': flow_id,
                    
                    # Processor identity (5 fields - added parent group)
                    'processor_id': proc.get('processor_id'),
                    'processor_name': proc.get('processor_name'),
                    'processor_type': proc.get('processor_type'),
                    'parent_group_id': proc.get('parent_group_id'),       # NEW
                    'parent_group_name': proc.get('parent_group_name'),   # NEW
                    
                    # Activity metrics (6 fields)
                    'flow_files_in': proc.get('flow_files_in', 0),
                    'bytes_in': proc.get('bytes_in', 0),
                    'flow_files_out': proc.get('flow_files_out', 0),
                    'bytes_out': proc.get('bytes_out', 0),
                    'tasks': proc.get('tasks', 0),
                    'run_status': proc.get('run_status', 'Unknown')
                })
            
            return flow_results
            
        except Exception as e:
            self.console.print(f"[red]ERROR[/red] Failed to analyze {flow_name}: {e}")
            flow_results['error'] = str(e)
            return flow_results
    
    def analyze_all_flows(self, flows_csv_path: str):
        """Analyze all flows from CSV."""
        self.console.print(f"\n[cyan]Multi-Flow Analysis Starting (PROCESSOR-LEVEL)...[/cyan]")
        self.console.print(f"  Server: {self.server}")
        self.console.print(f"  Timestamp: {self.snapshot_timestamp.strftime('%Y-%m-%d %H:%M:%S')}\n")
        
        # Read flows CSV
        try:
            flows_df = spark.read.csv(flows_csv_path, header=True)
            flows = flows_df.collect()
            
            self.console.print(f"[green]Found {len(flows)} flows to analyze[/green]\n")
            
        except Exception as e:
            self.console.print(f"[red]ERROR[/red] Failed to read CSV: {e}")
            raise
        
        # Analyze each flow
        with Progress(
            SpinnerColumn(),
            TextColumn("[progress.description]{task.description}"),
            BarColumn(),
            console=self.console
        ) as progress:
            task = progress.add_task("Analyzing flows...", total=len(flows))
            
            for flow in flows:
                flow_id = flow['id']
                flow_name = flow['flow_name']
                
                progress.update(task, description=f"Analyzing: {flow_name}")
                
                flow_results = self.analyze_flow(flow_id, flow_name)
                self.all_results.append(flow_results)
                
                # Display flow summary
                if 'error' not in flow_results:
                    self.console.print(
                        f"  [green]✓[/green] {flow_name}: {flow_results['processor_count']} processors"
                    )
                else:
                    self.console.print(
                        f"  [red]✗[/red] {flow_name}: {flow_results['error']}"
                    )
                
                progress.advance(task)
        
        # Display overall summary
        self.display_summary()
    
    def display_summary(self):
        """Display analysis summary."""
        total_processors = sum(r['processor_count'] for r in self.all_results if 'error' not in r)
        successful_flows = sum(1 for r in self.all_results if 'error' not in r)
        failed_flows = sum(1 for r in self.all_results if 'error' in r)
        
        self.console.print(f"\n[cyan]Overall Summary:[/cyan]")
        self.console.print(f"  Server: {self.server}")
        self.console.print(f"  Total flows: {len(self.all_results)}")
        self.console.print(f"  Successful: {successful_flows}")
        self.console.print(f"  Failed: {failed_flows}")
        self.console.print(f"  Total processors: {total_processors}")
        
        # Create summary table
        table = Table(title="\nFlow Analysis Summary")
        table.add_column("Flow Name", style="cyan")
        table.add_column("Processors", justify="right", style="yellow")
        table.add_column("Status", style="green")
        
        for result in self.all_results:
            status = "[red]Error[/red]" if 'error' in result else "[green]Success[/green]"
            table.add_row(
                result['flow_name'],
                str(result['processor_count']),
                status
            )
        
        self.console.print(table)
    
    def get_results_dataframe(self):
        """Convert all results to Spark DataFrame with 15-field processor schema (includes parent group)."""
        all_rows = []
        
        for flow_result in self.all_results:
            if 'error' not in flow_result:
                all_rows.extend(flow_result['processors'])
        
        if not all_rows:
            return None
        
        # Convert to list of tuples (15 fields)
        rows = [
            (
                row['snapshot_timestamp'],
                row['server'],
                row['flow_name'],
                row['process_group_id'],
                row['processor_id'],
                row['processor_name'],
                row['processor_type'],
                row.get('parent_group_id'),         # NEW
                row.get('parent_group_name'),       # NEW
                row.get('flow_files_in', 0),
                row.get('bytes_in', 0),
                row.get('flow_files_out', 0),
                row.get('bytes_out', 0),
                row.get('tasks', 0),
                row.get('run_status', 'Unknown')
            )
            for row in all_rows
        ]
        
        # Define schema (15 fields - processor level with parent group info)
        schema = StructType([
            # Metadata (4 fields)
            StructField("snapshot_timestamp", TimestampType(), False),
            StructField("server", StringType(), False),
            StructField("flow_name", StringType(), False),
            StructField("process_group_id", StringType(), False),
            
            # Processor identity (5 fields)
            StructField("processor_id", StringType(), True),
            StructField("processor_name", StringType(), False),
            StructField("processor_type", StringType(), True),
            StructField("parent_group_id", StringType(), True),      # NEW FIELD
            StructField("parent_group_name", StringType(), True),    # NEW FIELD
            
            # Activity metrics (6 fields)
            StructField("flow_files_in", LongType(), False),
            StructField("bytes_in", LongType(), False),
            StructField("flow_files_out", LongType(), False),
            StructField("bytes_out", LongType(), False),
            StructField("tasks", LongType(), False),
            StructField("run_status", StringType(), False)
        ])
        
        spark = SparkSession.builder.getOrCreate()
        return spark.createDataFrame(rows, schema)

console.print("[green]✓ MultiFlowAnalyzer class defined (with parent group info)![/green]")

In [ ]:
# Cell 6: Run Multi-Flow Analysis

console.print("\n[cyan]Starting Multi-Flow NiFi Analysis...[/cyan]\n")

# Connect to NiFi
console.print("[yellow]Connecting to NiFi...[/yellow]")
client = NiFiClient(
    base_url=CONFIG['nifi_url'],
    username=CONFIG['username'],
    password=CONFIG['password'],
    verify_ssl=CONFIG['verify_ssl']
)
console.print("[green]OK[/green] Connected successfully\n")

# Create analyzer and run analysis
analyzer = MultiFlowAnalyzer(client=client, server=CONFIG['server'])
analyzer.analyze_all_flows(CONFIG['flows_csv_path'])

# Cleanup
client.close()

console.print("\n[green]✓ Multi-flow analysis complete![/green]")

In [ ]:
# Cell 7: Save Snapshots to Delta Lake

if CONFIG['enable_snapshots']:
    console.print("\n[yellow]Saving snapshots to Delta Lake...[/yellow]")
    
    df = analyzer.get_results_dataframe()
    
    if df is not None:
        table_name = CONFIG['delta_table_path']
        
        # Check if table exists
        table_exists = spark.catalog._jcatalog.tableExists(table_name)
        
        if not table_exists:
            # First run: Create table
            console.print(f"[yellow]Table doesn't exist, creating: {table_name}[/yellow]")
            df.write \
                .format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .saveAsTable(table_name)
            console.print(f"[green]OK[/green] Table created successfully with 24-field connection-level schema")
        else:
            # Subsequent runs: Append data
            console.print(f"[yellow]Table exists, appending data to: {table_name}[/yellow]")
            df.write \
                .format("delta") \
                .mode("append") \
                .option("mergeSchema", "true") \
                .saveAsTable(table_name)
            console.print(f"[green]OK[/green] Data appended successfully")
        
        console.print(f"  Timestamp: {analyzer.snapshot_timestamp}")
        console.print(f"  Total rows written: {df.count()}")
        
        # Show sample
        console.print(f"\n[cyan]Sample data:[/cyan]")
        display(df.limit(10))
    else:
        console.print("[red]ERROR[/red] No data to save")
else:
    console.print("\n[yellow]Snapshots disabled[/yellow]")

# NOTE: If you need to manually drop the table to start fresh, run this in a separate cell:
# spark.sql(f"DROP TABLE IF EXISTS {CONFIG['delta_table_path']}")

In [ ]:
# Cell 8: Query Historical Snapshots (Connection-Level Analysis)

if CONFIG['enable_snapshots']:
    console.print("\n[cyan]Querying connection-level snapshots...[/cyan]\n")
    
    table_name = CONFIG['delta_table_path']
    
    try:
        # Show snapshots per flow and server
        console.print("[yellow]Snapshot count by server and flow:[/yellow]")
        spark.sql(f"""
            SELECT 
                server,
                flow_name,
                COUNT(DISTINCT snapshot_timestamp) as snapshots,
                COUNT(*) as total_connections,
                MAX(snapshot_timestamp) as last_snapshot
            FROM {table_name}
            GROUP BY server, flow_name
            ORDER BY server, flow_name
        """).show(truncate=False)
        
        # NEW: Find connections with high queue depth (backpressure detection)
        console.print("\n[yellow]Connections with queued flowfiles (backpressure):[/yellow]")
        spark.sql(f"""
            SELECT 
                server,
                flow_name,
                source_name,
                destination_name,
                MAX(queued_count) as max_queued_flowfiles,
                MAX(queued_bytes) as max_queued_bytes,
                MAX(percent_use_count) as max_percent_full
            FROM {table_name}
            WHERE queued_count > 0
            GROUP BY server, flow_name, source_name, destination_name
            ORDER BY max_queued_flowfiles DESC
            LIMIT 20
        """).show(truncate=False)
        
        # NEW: Identify connections approaching queue limits
        console.print("\n[yellow]Connections approaching queue limits (>50% full):[/yellow]")
        spark.sql(f"""
            SELECT 
                server,
                flow_name,
                source_name,
                destination_name,
                MAX(percent_use_count) as max_percent_full,
                MAX(queued_count) as max_queued_count
            FROM {table_name}
            WHERE percent_use_count > 50
            GROUP BY server, flow_name, source_name, destination_name
            ORDER BY max_percent_full DESC
            LIMIT 20
        """).show(truncate=False)
        
        # Find inactive connections (no flow for 7 days)
        console.print("\n[yellow]Inactive connections (no flowfiles for 7 days):[/yellow]")
        spark.sql(f"""
            WITH connection_activity AS (
                SELECT 
                    server,
                    flow_name,
                    source_name,
                    destination_name,
                    MAX(flow_files_out) - MIN(flow_files_out) as delta_flowfiles,
                    MIN(snapshot_timestamp) as first_snapshot,
                    MAX(snapshot_timestamp) as last_snapshot,
                    COUNT(DISTINCT snapshot_timestamp) as num_snapshots
                FROM {table_name}
                WHERE snapshot_timestamp >= current_date() - INTERVAL 7 DAYS
                GROUP BY server, flow_name, source_name, destination_name
            )
            SELECT 
                server,
                flow_name,
                source_name,
                destination_name,
                delta_flowfiles,
                num_snapshots
            FROM connection_activity
            WHERE delta_flowfiles = 0
            ORDER BY server, flow_name, source_name
            LIMIT 50
        """).show(truncate=False)
        
        # Aggregate to processor level (still possible!)
        console.print("\n[yellow]Inactive processors by flow (aggregated from connections):[/yellow]")
        spark.sql(f"""
            WITH processor_activity AS (
                SELECT 
                    server,
                    flow_name,
                    source_name as processor_name,
                    MAX(flow_files_out) - MIN(flow_files_out) as delta_flowfiles
                FROM {table_name}
                WHERE snapshot_timestamp >= current_date() - INTERVAL 7 DAYS
                GROUP BY server, flow_name, source_name
            )
            SELECT 
                server,
                flow_name,
                COUNT(*) as inactive_processor_count
            FROM processor_activity
            WHERE delta_flowfiles = 0
            GROUP BY server, flow_name
            ORDER BY server, inactive_processor_count DESC
        """).show(truncate=False)
        
        # NEW: Track queue growth over time
        console.print("\n[yellow]Queue depth trends (hourly averages):[/yellow]")
        spark.sql(f"""
            SELECT 
                DATE_TRUNC('hour', snapshot_timestamp) as hour,
                server,
                flow_name,
                source_name,
                destination_name,
                AVG(queued_count) as avg_queued_flowfiles,
                MAX(queued_count) as max_queued_flowfiles
            FROM {table_name}
            WHERE snapshot_timestamp >= current_date() - INTERVAL 1 DAYS
              AND queued_count > 0
            GROUP BY hour, server, flow_name, source_name, destination_name
            ORDER BY hour DESC, avg_queued_flowfiles DESC
            LIMIT 20
        """).show(truncate=False)
        
        # NEW: Bidirectional flow analysis
        console.print("\n[yellow]Flow balance (input vs output by connection):[/yellow]")
        spark.sql(f"""
            SELECT 
                server,
                flow_name,
                source_name,
                destination_name,
                SUM(flow_files_in) as total_flowfiles_in,
                SUM(flow_files_out) as total_flowfiles_out,
                SUM(flow_files_in) - SUM(flow_files_out) as net_change
            FROM {table_name}
            WHERE snapshot_timestamp >= current_date() - INTERVAL 7 DAYS
            GROUP BY server, flow_name, source_name, destination_name
            HAVING ABS(SUM(flow_files_in) - SUM(flow_files_out)) > 100
            ORDER BY ABS(net_change) DESC
            LIMIT 20
        """).show(truncate=False)
        
    except Exception as e:
        console.print(f"[red]ERROR[/red] Failed to query: {e}")
        import traceback
        traceback.print_exc()
else:
    console.print("\n[yellow]Snapshots disabled[/yellow]")

In [ ]:
# Cell 9: Export Results to CSV by Flow

console.print("\n[yellow]Exporting results to CSV...[/yellow]")

timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')

df = analyzer.get_results_dataframe()
if df is not None:
    pdf = df.toPandas()
    
    # Export overall summary
    output_path = f"/dbfs/nifi_analysis/all_flows_{timestamp_str}.csv"
    pdf.to_csv(output_path, index=False)
    console.print(f"[green]OK[/green] All flows exported to {output_path}")
    
    # Export per flow
    for flow_name in pdf['flow_name'].unique():
        flow_df = pdf[pdf['flow_name'] == flow_name]
        flow_path = f"/dbfs/nifi_analysis/{flow_name}_{timestamp_str}.csv"
        flow_df.to_csv(flow_path, index=False)
        console.print(f"  [green]✓[/green] {flow_name}: {len(flow_df)} processors")
    
    console.print(f"\n[cyan]Sample data:[/cyan]")
    display(pdf.head(10))
else:
    console.print("[red]ERROR[/red] No data to export")

---

## Updated Delta Table Schema (Connection-Level)

The Delta table now captures **ALL available fields** from NiFi Status API at the **connection level** (not processor level). This provides maximum granularity for analysis.

### 24 Total Fields

| Column | Type | Description |
|--------|------|-------------|
| **Metadata (4 fields)** | | |
| `snapshot_timestamp` | Timestamp | When the snapshot was captured |
| `server` | String | Server identifier (hostname, environment name) |
| `flow_name` | String | Flow name from CSV |
| `process_group_id` | String | NiFi process group ID |
| **Connection Identity (3 fields)** | | |
| `connection_id` | String | Connection UUID |
| `connection_name` | String | Connection name (often empty or "success") |
| `connection_group_id` | String | Parent process group ID |
| **Source Processor (2 fields)** | | |
| `source_id` | String | Source processor UUID |
| `source_name` | String | Source processor name |
| **Destination Processor (2 fields)** | | |
| `destination_id` | String | Destination processor UUID |
| `destination_name` | String | Destination processor name |
| **Flow Metrics - 5-minute window (6 fields)** | | |
| `flow_files_in` | Long | FlowFiles entering connection |
| `flow_files_out` | Long | FlowFiles leaving connection |
| `bytes_in` | Long | Bytes entering connection |
| `bytes_out` | Long | Bytes leaving connection |
| `input` | String | Formatted input stats (e.g., "1,250 (50.8 KB)") |
| `output` | String | Formatted output stats |
| **Queue Metrics - current state (4 fields)** | | |
| `queued_count` | Long | FlowFiles currently queued |
| `queued_bytes` | Long | Bytes currently queued |
| `queued` | String | Formatted queue stats |
| `queued_size` | String | Formatted queue size |
| **Status Indicators (2 fields)** | | |
| `percent_use_count` | Long | % of queue count threshold used |
| `percent_use_bytes` | Long | % of queue bytes threshold used |
| **Timestamps (1 field)** | | |
| `stats_last_refreshed` | String | When stats were last updated |

## Key Differences from Previous Version

**Before:** 9 fields, processor-level aggregation
- Only captured: flowFilesOut, bytesOut
- Aggregated connections by source processor
- Could not identify specific bottleneck connections
- No queue monitoring capability

**Now:** 24 fields, connection-level granularity
- Captures ALL 15+ fields from NiFi Status API
- Stores each connection separately
- Can identify exact bottleneck points
- Enables queue monitoring, backpressure detection, flow lineage

**Impact:** ~2x more rows (typical NiFi flow has 1-2 connections per processor), but unlocks powerful new analysis capabilities.

## New Analysis Capabilities

### 1. Backpressure Detection
Identify connections with high queue depth:
```sql
SELECT source_name, destination_name, MAX(queued_count) as max_queued
FROM main.default.nifi_processor_snapshots
WHERE queued_count > 100
GROUP BY source_name, destination_name
ORDER BY max_queued DESC;
```

### 2. Queue Limit Monitoring
Find connections approaching capacity:
```sql
SELECT source_name, destination_name, MAX(percent_use_count) as max_percent_full
FROM main.default.nifi_processor_snapshots
WHERE percent_use_count > 80
GROUP BY source_name, destination_name;
```

### 3. Bidirectional Flow Tracking
Compare input vs output to find imbalances:
```sql
SELECT source_name,
       SUM(flow_files_in) as total_in,
       SUM(flow_files_out) as total_out,
       SUM(flow_files_in) - SUM(flow_files_out) as net_change
FROM main.default.nifi_processor_snapshots
GROUP BY source_name
HAVING ABS(net_change) > 100;
```

### 4. Queue Growth Trends
Monitor queue depth over time:
```sql
SELECT DATE_TRUNC('hour', snapshot_timestamp) as hour,
       source_name, destination_name,
       AVG(queued_count) as avg_queued_flowfiles
FROM main.default.nifi_processor_snapshots
WHERE snapshot_timestamp >= current_date() - INTERVAL 1 DAYS
GROUP BY hour, source_name, destination_name
ORDER BY hour, avg_queued_flowfiles DESC;
```

### 5. Processor-Level Analysis (Still Possible!)
Aggregate connections to processor-level when needed:
```sql
WITH processor_activity AS (
    SELECT source_name,
           MAX(flow_files_out) - MIN(flow_files_out) as delta
    FROM main.default.nifi_processor_snapshots
    WHERE snapshot_timestamp >= current_date() - INTERVAL 7 DAYS
    GROUP BY source_name
)
SELECT * FROM processor_activity WHERE delta = 0;
```

### 6. Flow Path Lineage
Track data movement through the flow:
```sql
SELECT source_name, destination_name, 
       SUM(flow_files_out) as total_flowfiles
FROM main.default.nifi_processor_snapshots
WHERE snapshot_timestamp >= current_date() - INTERVAL 7 DAYS
GROUP BY source_name, destination_name
ORDER BY total_flowfiles DESC;
```

## Unity Catalog Configuration

The notebook uses Unity Catalog with 3-level naming:
- **Catalog**: `main` (default)
- **Schema**: `default` (default)
- **Table**: `nifi_processor_snapshots`
- **Full path**: `main.default.nifi_processor_snapshots`

You can customize this in Cell 3 by editing `delta_table_path`.

## CSV Format

Your `flows.csv` should look like:
```
id,flow_name
8c8677c4-29d6-3607-a32e-1234567890ab,Production_Data_Pipeline
abc-123-def-456-7890-abcdef123456,Development_Testing_Flow
xyz-789-ghi-012-3456-7890abcdef12,QA_Validation_Flow
```

Upload it to: `/dbfs/nifi_analysis/flows.csv`

## Server Identifier

The `server` field helps track data from multiple NiFi instances:
- Use hostname: `prod-nifi-01`, `dev-nifi-02`
- Use environment: `prod`, `dev`, `qa`, `staging`
- Use datacenter: `dc1-nifi`, `dc2-nifi`

This allows you to:
- Compare processor usage across environments
- Track migration from one server to another
- Aggregate metrics across multiple NiFi clusters

## Key Concepts

**Connection-Level Storage:**
- Each connection is stored as a separate row
- Preserves source → destination relationships
- Enables fine-grained debugging and analysis
- Can still aggregate to processor-level in queries

**Snapshot-based Analysis:**
- Each run captures a snapshot of flowfile counts at that moment
- Take snapshots every 5 minutes over a week
- Calculate deltas (MAX - MIN) to identify inactive connections
- Connection with delta = 0 means no flowfiles processed in that time period

**Why Connection-Level Instead of Processor-Level?**
- Identify which specific connection is bottlenecked
- Monitor queue depth per connection
- Track flow paths (source → destination lineage)
- More debugging capability with minimal storage overhead

**5-Minute Window:**
- NiFi Status API returns metrics aggregated over the last 5 minutes
- Running snapshots every 5 minutes captures distinct time windows
- Historical data retained for 24 hours (configurable in NiFi)

**Queue Metrics:**
- `queued_count`: Current number of FlowFiles waiting in connection
- `percent_use_count`: How full the queue is (approaching backpressure threshold)
- Helps identify bottlenecks before they cause performance issues

## Migration Notes

**IMPORTANT:** Running Cell 7 will DROP the existing table to start fresh with the new 24-field schema. This is necessary because:
1. Schema changed from 9 fields to 24 fields
2. Data model changed from processor-level to connection-level
3. Cannot merge old and new data structures

**Before running:** If you want to preserve old data, create a backup:
```python
spark.sql("CREATE TABLE main.default.nifi_processor_snapshots_backup AS SELECT * FROM main.default.nifi_processor_snapshots")
```